###  Unsupervised Outlier Detection using Local Correlation Integral (LOCI)

## Steps for implementation
#### Reading CSV
#### Data Description
#### Data Cleaning
#### Data Normalization
#### Features Selection
#### Applying ARIMA and LOCI

AutoRegressive Integrated Moving Average(ARIMA) captures a suite of different standard temporal structures in time series data.


 Local Correlation Integral(LOCI) is highly effective for detecting outliers and groups of outliers ( a.k.a.micro-clusters)

In [1]:
import numpy
numpy.version.version

'1.23.1'

In [27]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [28]:
# !pip uninstall numpy
# !pip install numpy==1.23.1

In [29]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
nRowsRead = None # Čita sve redove, moguće je pročitati samo određeni broj

df1 = pd.read_csv("./data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("./data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("./data/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("./data/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("./data/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("./data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("./data/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("./data/Wednesday-workingHours.pcap_ISCX.csv")

In [5]:
df = pd.concat([df1,df2])
del df1,df2
df = pd.concat([df,df3])
del df3
df = pd.concat([df,df4])
del df4
df = pd.concat([df,df5])
del df5
df = pd.concat([df,df6])
del df6
df = pd.concat([df,df7])
del df7
df = pd.concat([df,df8])
del df8

nRow, nCol = df.shape

In [6]:
print(f' rows : {nRow} columns : {nCol}')

 rows : 2830743 columns : 79


In [7]:
df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109,109,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,9174.311927,9174.311927,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52,52,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,19230.769230,19230.769230,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,6,6,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34,34,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,20,20,29411.764710,29411.764710,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,666666.666700,0.000000,6,6,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [8]:
list(df.columns)

[' Destination Port',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 'Flow Bytes/s',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Bwd PSH Flags',
 ' Fwd URG Flags',
 ' Bwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag Count',
 ' RST Flag Count',
 ' P

In [9]:
print(df[" Label"].unique())

['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']


In [10]:
df[' Label'].value_counts()

BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name:  Label, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2830743 entries, 0 to 692702
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max          

In [12]:
old_memory_usage = df.memory_usage().sum()
#change the variable types for low memory usage
#int64 to int32,,, float64 to float32
integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64": integer.append(i)
    else : f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2830743 entries, 0 to 692702
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int32  
 1    Flow Duration                int32  
 2    Total Fwd Packets            int32  
 3    Total Backward Packets       int32  
 4   Total Length of Fwd Packets   int32  
 5    Total Length of Bwd Packets  int32  
 6    Fwd Packet Length Max        int32  
 7    Fwd Packet Length Min        int32  
 8    Fwd Packet Length Mean       float32
 9    Fwd Packet Length Std        float32
 10  Bwd Packet Length Max         int32  
 11   Bwd Packet Length Min        int32  
 12   Bwd Packet Length Mean       float32
 13   Bwd Packet Length Std        float32
 14  Flow Bytes/s                  float32
 15   Flow Packets/s               float32
 16   Flow IAT Mean                float32
 17   Flow IAT Std                 float32
 18   Flow IAT Max          

In [13]:
# def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
#     nunique = df.nunique()
#     df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
#     nRow, nCol = df.shape
#     columnNames = list(df)
#     nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
#     plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
#     for i in range(min(nCol, nGraphShown)):
#         plt.subplot(nGraphRow, nGraphPerRow, i + 1)
#         columnDf = df.iloc[:, i]
#         if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
#             valueCounts = columnDf.value_counts()
#             valueCounts.plot.bar()
#         else:
#             columnDf.hist()
#         plt.ylabel('counts')
#         plt.xticks(rotation = 90)
#         plt.title(f'{columnNames[i]} (column {i})')
#     plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
#     plt.show()
# plotPerColumnDistribution(df, 79, 5)

In [14]:
new_memory_usage = df.memory_usage().sum()
old_vs_new = (old_memory_usage - new_memory_usage) / old_memory_usage * 100
print(f"%{old_vs_new} lower memory usage")

%48.75 lower memory usage


In [15]:
# drop one variable features 
one_variable_list = []
for i in df.columns:
    if df[i].value_counts().nunique() < 2:
        one_variable_list.append(i)
df.drop(one_variable_list,axis=1,inplace=True)
df.columns =  df.columns.str.strip()

In [16]:
# drop nan and infinite rows
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]

/tmp/ipykernel_673604/1114794877.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  df = df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]


In [17]:
# # merging similar classes with low instances
# df["Label"] = df["Label"].replace(["Web Attack � Brute Force","Web Attack � XSS","Web Attack � Sql Injection"],"Web Attack")

In [18]:
# drop duplicate rows
df =  df.drop_duplicates(keep="first")
df.reset_index(drop=True,inplace=True)

In [19]:
#feature reduction 
#dropping very high correlated features 
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df =  df.drop(to_drop, axis=1)
df.shape

/tmp/ipykernel_673604/862126478.py:3: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df.corr().abs()
/tmp/ipykernel_673604/862126478.py:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


(2498185, 48)

In [20]:
x = df.drop(["Label"],axis=1)
y = df["Label"]

In [21]:
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

#train and test split
X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.35,random_state=42,stratify=y)

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [25]:
#Fitting Models
from sklearn.naive_bayes import BernoulliNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
import time 
startNB = time.time()


# Treniranje Naive Baye modela
BNB_Classifier = BernoulliNB()
BNB_Classifier.fit(X_train, Y_train)
print("Vreme treniranja modela Naive Bayes", time.time()-startNB)

startDT = time.time()
# Treniranje Decision Tree modela
DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', splitter='best', random_state=0)
#Funkcija za merenje kvaliteta podele podrzava kriterijume „gini“, za nečistoću i „entropy“ za dobijanje informacija,
DTC_Classifier.fit(X_train, Y_train)
print("Vreme treniranja modela stablo odlucivanja", time.time()-startDT)

startKNN = time.time()
# Treniranje KNeighborsClassifier modela
KNN_Classifier = KNeighborsClassifier(n_jobs=-1, n_neighbors=3, metric='minkowski', algorithm='auto') 
#n_jobs=-1 znaci da koristi sve procesore,  algorithm='auto' sam primenjuje odgovarajuci algoritam na osnovu prosleđenih vrednosti, n_neighbors=3 uzima u obzir 3 najbliza suseda, minkowski udaljenost
KNN_Classifier.fit(X_train, Y_train); 
print("Vreme treniranja modela KNN", time.time()-startKNN)

Vreme treniranja modela Naive Bayes 6.744009494781494
Vreme treniranja modela stablo odlucivanja 39.26307964324951
Vreme treniranja modela KNN 1.8503162860870361


In [26]:
#Evaluacija modela
from sklearn import metrics

models = []
models.append(('Naive Baye Classifier', BNB_Classifier))
# models.append(('Decision Tree Classifier', DTC_Classifier))
models.append(('KNeighborsClassifier', KNN_Classifier))



for i, v in models:
    scores = cross_val_score(v, X_train, Y_train, cv=7)
    accuracy = metrics.accuracy_score(Y_train, v.predict(X_train))
    confusion_matrix = metrics.confusion_matrix(Y_train, v.predict(X_train))
    classification = metrics.classification_report(Y_train, v.predict(X_train))
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()


============================== Naive Baye Classifier Model Evaluation ==============================

Cross Validation Mean Score:
 0.7998010866822778

Model Accuracy:
 0.7996828466209309

Confusion matrix:
 [[1075917    2885   31316   49072   10553   18726   30752    6893    2664
    66974   20507    4400   10033      74   16322]
 [    500       0       0       0       0       0       1     765       0
        0       0       0       0       0       0]
 [  17567       0   57631       0    7996       5       3       7       0
        0       0       0       0       0       0]
 [    346       0       0    4544     219     217    1306       0       0
       54       0       0       0       0       0]
 [  11971       0      74    8280   91658     190     126      47       0
        1       0       0       1       0       2]
 [    602       0       1       5       0    2453     324       0       0
        8       0       0       3       0       2]
 [   1262       0       1       4       0

In [ ]:
#definisanje funkcije za prikaz matrice tacnosti
def plot_confusion_matrix(cm,
                          title,
                          cmap=None,
                          target_names=None,
                          normalize=False
                         ):
    
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.grid(False)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()
    plt.savefig(title, bbox_inches='tight', dpi=300)


In [ ]:
#Rezultat
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import scikitplot as skplt

models = []
models.append(('Naive Baye Classifier', BNB_Classifier))
models.append(('Decision Tree Classifier', DTC_Classifier))
models.append(('KNeighborsClassifier', KNN_Classifier))

for i, v in models:
    
    accuracy = metrics.accuracy_score(Y_test, v.predict(X_test))
    confusion_matrix = metrics.confusion_matrix(Y_test, v.predict(X_test))
    classification = metrics.classification_report(Y_test, v.predict(X_test))
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()
    plot_confusion_matrix(cm=confusion_matrix ,
                      title        = i)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from random import random

# fit model
model = ARIMA(X_train[" Fwd Packet Length Max"][0:1000], order=(1, 1, 1))
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(1000, 1000, type='levels')
print(yhat)

print(model_fit.forecast())

In [ ]:
from pyod.models.loci import LOCI

clf_name = 'LOCI'
clf = LOCI()
clf.fit(X_train[:600])
y_test_pred = clf.predict(X_test[:300])  # outlier labels (0 or 1)
print("Total Samples : ",len(y_test_pred))
print( "Inliers : ",len(y_test_pred[y_test_pred == 0]))
print( "Outliers : ",len(y_test_pred[y_test_pred == 1]))

In [ ]:
from pyod.models.cblof import CBLOF
clf_name = 'CBLOF'
clf = CBLOF(contamination=0.09)
clf.fit(X_train)
# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
# evaluate and print the results
#print("\nOn Training Data:" + y_train_scores)
#evaluate_print(y_train_scores)
#print("\nOn Test Data:" + y_test_scores)
#evaluate_print(clf_name, y_test,y_test_scores)
print("Total Samples : ",len(y_test_pred))
print( "Inliers : ",len(y_test_pred[y_test_pred == 0]))
print( "Outliers : ",len(y_test_pred[y_test_pred == 1]))

In [ ]:
from pyod.models.hbos import HBOS
clf_name = 'HBOS'
clf = HBOS(contamination=0.09)
clf.fit(X_train)
y_test_pred = clf.predict(X_test)
#y_test_arr=y_test.as_matrix()
print("Total Samples : ",len(y_test_pred))
print( "Inliers : ",len(y_test_pred[y_test_pred == 0]))
print( "Outliers : ",len(y_test_pred[y_test_pred == 1]))


In [ ]:
from pyod.models.auto_encoder import AutoEncoder

clf2 = AutoEncoder(hidden_neurons =[14, 10,2, 10, 14], epochs=20)
clf2.fit(X_train)

# Predict the anomaly scores


In [1]:
#Importing Necessary Modules
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split     # import module for train test split
from pyod.models.knn import KNN
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.loci import LOCI
from pyod.utils.data import evaluate_print

2023-03-03 12:14:24.017946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 12:14:24.378427: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-03 12:14:24.435835: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-03 12:14:24.435854: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

### Dataset link https://www.kaggle.com/datasets/cicdataset/cicids2017

### Reading CSV

In [4]:
# df1=pd.read_csv("../../Datasets/MachineLearningCVE/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
# df2=pd.read_csv("../../Datasets/MachineLearningCVE/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df1=pd.read_csv("./data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("./data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")

#df3=pd.read_csv("../Datasets/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
#df4=pd.read_csv("../Datasets/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
#df5=pd.read_csv("../Datasets/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
#df6=pd.read_csv("../Datasets/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
#df7=pd.read_csv("../Datasets/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
#df8=pd.read_csv("../Datasets/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")


In [11]:
def preprosess(df):

    #remove rows with any values that are not finite
    df = df[np.isfinite(df).all(1)]
    
    df=df.dropna( axis=0, how='any')
    df=df.replace(',,', np.nan, inplace=False)
    df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

    df.replace("Infinity", 0, inplace=True)
    df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
    df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
    df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
    df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)
    df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
    df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
    df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)
    df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
    df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
    df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)
    df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
    df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
    df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)
    df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
    df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
    df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)
    df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
    df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
    df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)
    df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
    df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
    df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)
    df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
    df[" CWE Flag Count"].replace(",,", 0, inplace=True)
    df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)
    df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
    df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
    df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)
    df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
    df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
    df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)
    df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
    df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
    df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

    df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
    df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")
    df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
    df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
    df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
    df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
    df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
    df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
    df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
    df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
    df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
    df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")
    pd.set_option('display.max_rows', df.shape[0])
    df.replace('Infinity',0.0, inplace=True)
    df.replace('NaN',0.0, inplace=True)

    integer = []
    f = []
    for i in df.columns[:-1]:
        if df[i].dtype == "int64": integer.append(i)
        else : f.append(i)

    df[integer] = df[integer].astype("int32")
    df[f] = df[f].astype("float32")

    return df


In [13]:
df = pd.read_csv("./data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")


df_ddos=df[df[' Label']=='DDoS']
df = df_ddos.drop(columns=[' Label'], axis=1, inplace=False)
vr = df.head(1)
t = preprosess(vr)
t.head(1)

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
18883,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0


In [3]:

df=pd.concat([df1,df2])
#df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [4]:
del df1,df2
#del df1,df2,df3,df4,df5,df6,df7,df8


In [5]:
# category A attacks
df_bot=df[df[' Label']=='Bot']
df_ddos=df[df[' Label']=='DDoS']
df_golden_eye=df[df[' Label']=='DoS GoldenEye']
df_hulk=df[df[' Label']=='DoS Hulk']
df_ssh_patator=df[df[' Label']=='SSH-Patator']
df_ftp_patator=df[df[' Label']=='FTP-Patator']
df_infiltration=df[df[' Label']=='Infiltration']

In [6]:
# get_all_users# benign data
df_benign=df[df[' Label']=='BENIGN']

In [7]:
df_cat_A=pd.concat([df_bot,df_ddos,df_golden_eye,df_hulk,df_ssh_patator,df_ftp_patator,df_infiltration, df_benign])
df_cat_A.groupby(' Label').first()


,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,
BENIGN,54865,3,2,0,12,0,6,6,6.000000,0.000000,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
DDoS,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0


In [8]:
del df_bot,df_ddos,df_golden_eye,df_hulk,df_ssh_patator,df_ftp_patator,df_infiltration, df_benign

In [9]:
df=df_cat_A.copy()

In [10]:
del df_cat_A

In [11]:
df.groupby(' Label').first()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
Label,,,,,,,,,,,,,,,,,,,,,
BENIGN,54865,3,2,0,12,0,6,6,6.000000,0.000000,...,1,20,0.0,0.0,0,0,0.0,0.0,0,0
DDoS,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,...,2,20,0.0,0.0,0,0,0.0,0.0,0,0


### Data Description

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353282 entries, 18883 to 286466
Data columns (total 79 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Destination Port             353282 non-null  int64  
 1    Flow Duration                353282 non-null  int64  
 2    Total Fwd Packets            353282 non-null  int64  
 3    Total Backward Packets       353282 non-null  int64  
 4   Total Length of Fwd Packets   353282 non-null  int64  
 5    Total Length of Bwd Packets  353282 non-null  int64  
 6    Fwd Packet Length Max        353282 non-null  int64  
 7    Fwd Packet Length Min        353282 non-null  int64  
 8    Fwd Packet Length Mean       353282 non-null  float64
 9    Fwd Packet Length Std        353282 non-null  float64
 10  Bwd Packet Length Max         353282 non-null  int64  
 11   Bwd Packet Length Min        353282 non-null  int64  
 12   Bwd Packet Length Mean       353282 non

In [13]:
df.describe()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,353282.000000,3.532820e+05,353282.000000,353282.000000,353282.000000,3.532820e+05,353282.000000,353282.000000,353282.000000,353282.000000,...,353282.000000,353282.000000,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05,3.532820e+05
mean,8317.536203,1.470301e+07,5.473658,5.324557,789.085334,5.998437e+03,409.347516,25.766543,124.442551,157.246116,...,3.515919,22.886063,1.457166e+05,2.687304e+04,1.960644e+05,1.310762e+05,7.859771e+06,2.357610e+06,9.528822e+06,6.165548e+06
std,18974.050876,3.159391e+07,21.336977,30.756145,3091.009774,5.552206e+04,1526.654248,132.061381,412.573844,650.176230,...,16.529588,5.380668,7.644236e+05,2.895786e+05,9.742138e+05,7.354875e+05,1.929626e+07,1.039089e+07,2.328394e+07,1.762792e+07
min,0.000000,-1.300000e+01,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,53.000000,2.367700e+04,2.000000,1.000000,26.000000,6.000000e+00,6.000000,0.000000,6.000000,0.000000,...,1.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,80.000000,4.694965e+05,3.000000,2.000000,44.000000,1.500000e+02,20.000000,6.000000,8.666667,0.000000,...,2.000000,20.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,443.000000,6.696332e+06,5.000000,5.000000,86.000000,1.160100e+04,47.000000,31.000000,44.000000,10.263203,...,4.000000,20.000000,9.920000e+02,0.000000e+00,9.920000e+02,9.910000e+02,6.105852e+06,0.000000e+00,6.107426e+06,5.892240e+06
max,65532.000000,1.199999e+08,3119.000000,3635.000000,232349.000000,7.150819e+06,13929.000000,1472.000000,3867.000000,6692.644993,...,2056.000000,60.000000,1.100000e+08,7.050000e+07,1.100000e+08,1.100000e+08,1.200000e+08,7.050000e+07,1.200000e+08,1.200000e+08


In [14]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

### Data Cleaning

In [15]:
#df2=df[df.columns[7:-1]]
#df2[' Label']=df[' Label']
#df=df2
#df2=[]
#del df2

In [16]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [17]:
len(df.columns)

79

In [18]:
df=df.dropna( axis=0, how='any')
df=df.replace(',,', np.nan, inplace=False)
df=df.drop(columns=[' Fwd Header Length.1'], axis=1, inplace=False)

In [19]:
df.replace("Infinity", 0, inplace=True)
df['Flow Bytes/s'].replace("Infinity", 0,inplace=True)
df[" Flow Packets/s"].replace("Infinity", 0, inplace=True)
df[" Flow Packets/s"].replace(np.nan, 0, inplace=True)
df['Flow Bytes/s'].replace(np.nan, 0,inplace=True)


df["Bwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df["Bwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)

df[" Bwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df[" Bwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df[" Bwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Bulk Rate"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(",,", 0, inplace=True)
df[" Fwd Avg Bulk Rate"].replace(np.nan, 0, inplace=True)


df[" Fwd Avg Packets/Bulk"].replace("Infinity", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(",,", 0, inplace=True)
df[" Fwd Avg Packets/Bulk"].replace(np.nan, 0, inplace=True)


df["Fwd Avg Bytes/Bulk"].replace("Infinity", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(",,", 0, inplace=True)
df["Fwd Avg Bytes/Bulk"].replace(np.nan, 0, inplace=True)


df[" CWE Flag Count"].replace("Infinity", 0, inplace=True)
df[" CWE Flag Count"].replace(",,", 0, inplace=True)
df[" CWE Flag Count"].replace(np.nan, 0, inplace=True)

df[" Bwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd URG Flags"].replace(",,", 0, inplace=True)
df[" Bwd URG Flags"].replace(np.nan, 0, inplace=True)

df[" Bwd PSH Flags"].replace("Infinity", 0, inplace=True)
df[" Bwd PSH Flags"].replace(",,", 0, inplace=True)
df[" Bwd PSH Flags"].replace(np.nan, 0, inplace=True)

df[" Fwd URG Flags"].replace("Infinity", 0, inplace=True)
df[" Fwd URG Flags"].replace(",,", 0, inplace=True)
df[" Fwd URG Flags"].replace(np.nan, 0, inplace=True)

In [20]:
df["Flow Bytes/s"]=df["Flow Bytes/s"].astype("float64")
df[' Flow Packets/s']=df[" Flow Packets/s"].astype("float64")

In [21]:
df['Bwd Avg Bulk Rate']=df["Bwd Avg Bulk Rate"].astype("float64")
df[' Bwd Avg Packets/Bulk']=df[" Bwd Avg Packets/Bulk"].astype("float64")
df[' Bwd Avg Bytes/Bulk']=df[" Bwd Avg Bytes/Bulk"].astype("float64")
df[' Fwd Avg Bulk Rate']=df[" Fwd Avg Bulk Rate"].astype("float64")
df[' Fwd Avg Packets/Bulk']=df[" Fwd Avg Packets/Bulk"].astype("float64")
df['Fwd Avg Bytes/Bulk']=df["Fwd Avg Bytes/Bulk"].astype("float64")
df[' CWE Flag Count']=df[" CWE Flag Count"].astype("float64")
df[' Bwd URG Flags']=df[" Bwd URG Flags"].astype("float64")
df[' Bwd PSH Flags']=df[" Bwd PSH Flags"].astype("float64")
df[' Fwd URG Flags']=df[" Fwd URG Flags"].astype("float64")

In [22]:
pd.set_option('display.max_rows', df.shape[0])

In [23]:
df.replace('Infinity',0.0, inplace=True)

In [24]:
df.replace('NaN',0.0, inplace=True)

In [25]:
df.columns

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [26]:
df_benign=df[df[' Label']=='BENIGN']
df_benign.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [27]:
df_ddos=df[df[' Label']=='DDoS']
df_ddos.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
18883,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18884,80,4421382,4,0,24,0,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18885,80,1083538,3,6,26,11601,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18886,80,80034360,8,4,56,11601,20,0,7.000000,5.656854,...,20,939.0,0.0,939,939,39300000.0,44200000.0,70600000,8072664,DDoS
18887,80,642654,3,6,26,11607,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS


In [28]:


X=df_benign[df_benign.columns[0:-1]]
y=df_benign[df_benign.columns[-1]]

del df

### Data Normalization

In [29]:
from scipy import stats


In [30]:


cols = list(X.columns)
for col in cols:
    X[col] = stats.zscore(X[col])
    print(col)



/tmp/ipykernel_1321324/481651549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = stats.zscore(X[col])


 Destination Port
 Flow Duration
 Total Fwd Packets
 Total Backward Packets
Total Length of Fwd Packets
 Total Length of Bwd Packets
 Fwd Packet Length Max
 Fwd Packet Length Min
 Fwd Packet Length Mean
 Fwd Packet Length Std
Bwd Packet Length Max
 Bwd Packet Length Min
 Bwd Packet Length Mean
 Bwd Packet Length Std
Flow Bytes/s
 Flow Packets/s
 Flow IAT Mean
 Flow IAT Std


/home/pc/.local/lib/python3.8/site-packages/numpy/core/_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


 Flow IAT Max
 Flow IAT Min
Fwd IAT Total
 Fwd IAT Mean
 Fwd IAT Std
 Fwd IAT Max
 Fwd IAT Min
Bwd IAT Total
 Bwd IAT Mean
 Bwd IAT Std
 Bwd IAT Max
 Bwd IAT Min
Fwd PSH Flags
 Bwd PSH Flags
 Fwd URG Flags
 Bwd URG Flags
 Fwd Header Length
 Bwd Header Length
Fwd Packets/s
 Bwd Packets/s
 Min Packet Length
 Max Packet Length
 Packet Length Mean
 Packet Length Std
 Packet Length Variance
FIN Flag Count
 SYN Flag Count
 RST Flag Count
 PSH Flag Count
 ACK Flag Count
 URG Flag Count
 CWE Flag Count
 ECE Flag Count
 Down/Up Ratio
 Average Packet Size
 Avg Fwd Segment Size
 Avg Bwd Segment Size
Fwd Avg Bytes/Bulk
 Fwd Avg Packets/Bulk
 Fwd Avg Bulk Rate
 Bwd Avg Bytes/Bulk
 Bwd Avg Packets/Bulk
Bwd Avg Bulk Rate
Subflow Fwd Packets
 Subflow Fwd Bytes
 Subflow Bwd Packets
 Subflow Bwd Bytes
Init_Win_bytes_forward
 Init_Win_bytes_backward
 act_data_pkt_fwd
 min_seg_size_forward
Active Mean
 Active Std
 Active Max
 Active Min
Idle Mean
 Idle Std
 Idle Max
 Idle Min


In [31]:
X.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,1.864799,-0.421474,-0.151613,-0.169201,-0.317394,-0.075181,-0.334656,-0.20243,-0.366483,-0.303627,...,-0.127972,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
1,1.873217,-0.421471,-0.189113,-0.143173,-0.318971,-0.075095,-0.334656,-0.20243,-0.366483,-0.303627,...,-0.176358,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
2,1.873261,-0.421473,-0.189113,-0.143173,-0.318971,-0.075095,-0.334656,-0.20243,-0.366483,-0.303627,...,-0.176358,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
3,1.480466,-0.421473,-0.189113,-0.143173,-0.318971,-0.075095,-0.334656,-0.20243,-0.366483,-0.303627,...,-0.176358,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434
4,1.864710,-0.421474,-0.151613,-0.169201,-0.317394,-0.075181,-0.334656,-0.20243,-0.366483,-0.303627,...,-0.127972,-0.734378,-0.152339,-0.116509,-0.181213,-0.130066,-0.343143,-0.159248,-0.341465,-0.310434


### Feature Selection

In [32]:
features=[" Fwd Packet Length Max"," Flow IAT Std"," Fwd Packet Length Std" ,"Fwd IAT Total",' Flow Duration', " Fwd Packet Length Mean", " Total Length of Bwd Packets", "Total Length of Fwd Packets", " Flow IAT Mean", " Bwd Packet Length Mean",  " Flow IAT Max", " Bwd Packet Length Std", ' Total Fwd Packets', ' Total Backward Packets']

In [33]:
X=X[features].copy()
X.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets
0,-0.334656,-0.353944,-0.303627,-0.397576,-0.421474,-0.366483,-0.075181,-0.317394,-0.316342,-0.528766,-0.364526,-0.413824,-0.151613,-0.169201
1,-0.334656,-0.353944,-0.303627,-0.397576,-0.421471,-0.366483,-0.075095,-0.318971,-0.316306,-0.506577,-0.364520,-0.413824,-0.189113,-0.143173
2,-0.334656,-0.353944,-0.303627,-0.397576,-0.421473,-0.366483,-0.075095,-0.318971,-0.316325,-0.506577,-0.364523,-0.413824,-0.189113,-0.143173
3,-0.334656,-0.353944,-0.303627,-0.397576,-0.421473,-0.366483,-0.075095,-0.318971,-0.316331,-0.506577,-0.364524,-0.413824,-0.189113,-0.143173
4,-0.334656,-0.353944,-0.303627,-0.397576,-0.421474,-0.366483,-0.075181,-0.317394,-0.316342,-0.528766,-0.364526,-0.413824,-0.151613,-0.169201


In [34]:
len(X.columns)

14

### Applying Algorithms

In [35]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, random_state=10)

In [36]:
from statsmodels.tsa.arima.model import ARIMA
from random import random

In [125]:
# fit model
model = ARIMA(X_train[" Fwd Packet Length Max"][0:1000], order=(1, 1, 1))
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(1000, 1000, type='levels')
print(yhat)

/home/pc/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/pc/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/pc/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


1000   -0.222787
dtype: float64


/home/pc/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [127]:
print(model_fit.forecast())

1000   -0.222787
dtype: float64


/home/pc/.local/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:
model_fit.predict(typ)

In [38]:
X_train.dtypes

 Fwd Packet Length Max          float64
 Flow IAT Std                   float64
 Fwd Packet Length Std          float64
Fwd IAT Total                   float64
 Flow Duration                  float64
 Fwd Packet Length Mean         float64
 Total Length of Bwd Packets    float64
Total Length of Fwd Packets     float64
 Flow IAT Mean                  float64
 Bwd Packet Length Mean         float64
 Flow IAT Max                   float64
 Bwd Packet Length Std          float64
 Total Fwd Packets              float64
 Total Backward Packets         float64
dtype: object

In [39]:
X_train.head()

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets
260088,-0.337856,-0.322609,-0.303627,-0.356858,-0.376049,-0.378370,-0.074922,-0.320548,-0.218235,-0.506577,-0.337609,-0.413824,-0.114114,-0.091118
44296,-0.337856,-0.353944,-0.303627,-0.397575,-0.421474,-0.378370,-0.075181,-0.320548,-0.316342,-0.528766,-0.364525,-0.413824,-0.151613,-0.169201
231937,-0.318659,-0.339054,-0.303627,-0.397576,-0.416507,-0.307047,-0.071492,-0.301623,-0.298462,-0.055408,-0.356017,-0.413824,-0.151613,-0.117146
155894,-0.334656,-0.300875,-0.303627,-0.397576,-0.395412,-0.366483,-0.074663,-0.318971,-0.269437,-0.506577,-0.321353,-0.413824,-0.189113,-0.013036
4638,-0.312260,-0.353944,-0.303627,-0.397576,-0.420956,-0.283273,-0.074490,-0.307931,-0.310744,-0.351257,-0.363638,-0.413824,-0.189113,-0.143173


In [40]:
# y_test_arr=y_test.as_matrix()
# y_test_arr=y_test.asfreq

In [41]:
# y_test_arr

#### PYOD

#### LOCI

In [42]:
X_test.shape

(45048, 14)

In [43]:
clf_name = 'LOCI'
clf = LOCI()

In [44]:
clf.fit(X_train[:600])

LOCI(alpha=0.5, contamination=0.1, k=None)

In [45]:
# get the prediction on the test data
y_test_pred = clf.predict(X_test[:300])  # outlier labels (0 or 1)


In [46]:
print("Total Samples : ",len(y_test_pred))
print( "Inliers : ",len(y_test_pred[y_test_pred == 0]))
print( "Outliers : ",len(y_test_pred[y_test_pred == 1]))

Total Samples :  300
Inliers :  202
Outliers :  98


In [52]:
y_test_pred

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets
79221,-0.334656,0.524563,-0.303627,-0.397576,-0.041927,-0.366483,-0.074749,-0.318971,0.503384,-0.506577,0.286260,-0.413824,-0.189113,-0.039063
2590,-0.337856,-0.353944,-0.303627,-0.397576,-0.421312,-0.378370,-0.075181,-0.320548,-0.314585,-0.528766,-0.364247,-0.413824,-0.189113,-0.143173
59511,-0.334656,-0.353944,-0.303627,-0.397570,-0.421469,-0.366483,-0.075181,-0.317394,-0.316284,-0.528766,-0.364516,-0.413824,-0.151613,-0.169201
59503,-0.334656,-0.353944,-0.303627,-0.397570,-0.421469,-0.366483,-0.075181,-0.317394,-0.316282,-0.528766,-0.364516,-0.413824,-0.151613,-0.169201
284623,-0.317593,-0.353929,-0.303627,-0.397576,-0.421469,-0.303085,-0.072242,-0.300572,-0.316324,-0.151559,-0.364517,-0.413824,-0.151613,-0.117146


In [47]:
# y_test_pred = clf.predict(X_Anomaly)

In [48]:
# print("Total Samples : ",len(y_test_pred))
# print( "Inliers : ",len(y_test_pred[y_test_pred == 0]))
# print( "Outliers : ",len(y_test_pred[y_test_pred == 1]))

In [63]:
dff1 = X_test[:300]
dff2 = pd.DataFrame(y_test_pred, columns=['outlier'])

In [104]:
frame = { 'actual_label': y[:300]}
  
y_df = pd.DataFrame(frame)

In [105]:
dff1.index = dff2.index = y_df.index

In [106]:
y_df.head(5)

,actual_label
0,BENIGN
1,BENIGN
2,BENIGN
3,BENIGN
4,BENIGN


In [113]:
pd.concat([dff1, dff2, y_df], axis=1).head(5)

,Fwd Packet Length Max,Flow IAT Std,Fwd Packet Length Std,Fwd IAT Total,Flow Duration,Fwd Packet Length Mean,Total Length of Bwd Packets,Total Length of Fwd Packets,Flow IAT Mean,Bwd Packet Length Mean,Flow IAT Max,Bwd Packet Length Std,Total Fwd Packets,Total Backward Packets,outlier,actual_label
0,-0.334656,0.524563,-0.303627,-0.397576,-0.041927,-0.366483,-0.074749,-0.318971,0.503384,-0.506577,0.286260,-0.413824,-0.189113,-0.039063,1,BENIGN
1,-0.337856,-0.353944,-0.303627,-0.397576,-0.421312,-0.378370,-0.075181,-0.320548,-0.314585,-0.528766,-0.364247,-0.413824,-0.189113,-0.143173,0,BENIGN
2,-0.334656,-0.353944,-0.303627,-0.397570,-0.421469,-0.366483,-0.075181,-0.317394,-0.316284,-0.528766,-0.364516,-0.413824,-0.151613,-0.169201,0,BENIGN
3,-0.334656,-0.353944,-0.303627,-0.397570,-0.421469,-0.366483,-0.075181,-0.317394,-0.316282,-0.528766,-0.364516,-0.413824,-0.151613,-0.169201,0,BENIGN
4,-0.317593,-0.353929,-0.303627,-0.397576,-0.421469,-0.303085,-0.072242,-0.300572,-0.316324,-0.151559,-0.364517,-0.413824,-0.151613,-0.117146,0,BENIGN
